In [ ]:
#The point of this is to parse clipper output as well as the DMSO TAM normalized data in the file:
#hnM_DMSO_TAM_merge_final_union.bg into the format required by rMAPS

In [ ]:
#first import the CLIPMap.test.peaks.txt to see what it looks like
#Note - Paper says at least 7 columns, so just use FDR

In [ ]:
#Note - it is also probably worthwhile to split clusters from the DMSO TAM normalized data to clusters
#that are the same between both samples as well as different between both samples. 

#I should also write a script to compare rMATS data between both conditions so that I can then filter common
#and unique events. I think I already wrote a script to do this via splice_ids. Is this sufficient, or should I
#compare by pybedtools merge by merging on variable exon and taking the average of the dPSI?

#All this issue with the novel splice sites could be eliminated by re-running rMATS with the new version and not
#using novel splice sites, but be sure to use the low cutoff

In [89]:
import pandas as pd
import numpy as np
import pybedtools as pbt

In [3]:
ls

all_possible_hnRNPM_pentamers
CLIPMap.test.peaks.txt
parse_clipper_to_rMAPS.ipynb
rMATS.3.0.9_0.0001_cutoff_CC1_CC2_motif_analysis/
rMATS.3.0.9_0.05_cutoff_CC1_CC2_motif_analysis/
rMATS.3.2.2_CC1_CC2_motif_analysis/
rMATS.3.2.2_CC1_CC2_TAM_clip_analysis /
rMATS.3.2.2_CC3_CC4_motif_analysis/
rMATS.3.2.2_CC3_CC4_TAM_clip_analysis/
rMATS.3.2.2_HH5_HH6_DMSO_clip_analysis/
rMATS.3.2.2_HH5_HH6_motif_analysis/


In [4]:
example_df = pd.read_csv('CLIPMap.test.peaks.txt',sep='\t')

In [6]:
example_df.head()

,#chr,start,stop,cluster_name,read_count,strand,p_value,fdr_eight_column_not_required
0,chr1,564464,564812,cluster_11,358,+,0.000000,0.000000
1,chr1,564870,564984,cluster_15,14,+,0.013144,0.020657
2,chr1,564997,565449,cluster_17,741,+,0.000000,0.000000
3,chr1,565496,566016,cluster_22,546,+,0.000000,0.000000
4,chr1,566057,566295,cluster_28,213,+,0.000000,0.000000


In [75]:
#now load from the clipper directory for DMSO for example
#/media/sam/Data1/hnRNPM_iCLIP/iCLIP_analysis/getting_clipper_to_work/clipper_runs/hnM_DMSO_clipper/
clipper_header = ['chr','start','stop','cluster_name','fdr','strand','apex_start','apex_stop']
rMAPS_header = ['chr','start','stop','cluster_name','read_count','strand','fdr']
dmso_clipper_df = pd.read_csv('/media/sam/Data1/hnRNPM_iCLIP/iCLIP_analysis/getting_clipper_to_work/clipper_runs/hnM_DMSO_clipper/dmso_clipper_out',sep='\t',header=None,names=clipper_header)

#now take this and parse into the format above
#get the read counts from the cluster name as a new column
dmso_clipper_df['read_count'] = dmso_clipper_df.cluster_name.apply(lambda x: x.split('_')[-1])
#now reorder the columns to match the rMAPS_header and save as a file
dmso_clipper_df = dmso_clipper_df[rMAPS_header]
dmso_clipper_df.to_csv('hnM_DMSO_clipper_out_rMAPS',sep='\t',header=False,index=False)
dmso_clipper_df.head()


,chr,start,stop,cluster_name,read_count,strand,fdr
0,chr1,251624,251660,ENSG00000228463.4_0_13,13,-,2.013922e-11
1,chr1,564580,564587,ENSG00000225972.1_0_7,7,+,2.627611e-04
2,chr1,564587,564609,ENSG00000225972.1_1_7,7,+,7.067030e-03
3,chr1,565338,565345,ENSG00000225630.1_0_7,7,+,2.343836e-04
4,chr1,565345,565367,ENSG00000225630.1_1_7,7,+,5.980699e-03


In [74]:
#now do the same thing but for tam clipper
#/media/sam/Data1/hnRNPM_iCLIP/iCLIP_analysis/getting_clipper_to_work/clipper_runs/hnM_TAM_clipper/
clipper_header = ['chr','start','stop','cluster_name','fdr','strand','apex_start','apex_stop']
rMAPS_header = ['chr','start','stop','cluster_name','read_count','strand','fdr']
tam_clipper_df = pd.read_csv('/media/sam/Data1/hnRNPM_iCLIP/iCLIP_analysis/getting_clipper_to_work/clipper_runs/hnM_TAM_clipper/tam_clipper_out',\
                              sep='\t',header=None,names=clipper_header,)
tam_clipper_df.head()
#now take this and parse into the format above
#get the read counts from the cluster name as a new column
tam_clipper_df['read_count'] = tam_clipper_df.cluster_name.apply(lambda x: x.split('_')[-1])
#now reorder the columns to match the rMAPS_header and save as a file
tam_clipper_df = tam_clipper_df[rMAPS_header]
tam_clipper_df['cluster_name'] = tam_clipper_df.cluster_name.apply(lambda x: x.replace('.','_'))
tam_clipper_df.to_csv('hnM_TAM_clipper_out_rMAPS',sep='\t',index=False,header=False)
tam_clipper_df.head()

,chr,start,stop,cluster_name,read_count,strand,fdr
0,chr1,564707,564711,ENSG00000225972_1_0_17,17,+,2.413035e-10
1,chr1,564711,564740,ENSG00000225972_1_1_33,33,+,2.817336e-09
2,chr1,564740,564778,ENSG00000225972_1_2_54,54,+,2.589563e-18
3,chr1,565230,565233,ENSG00000225630_1_0_7,7,+,1.202675e-04
4,chr1,565242,565254,ENSG00000225630_1_1_7,7,+,3.194029e-03


In [109]:
#make this into a function
#There are clipper windows that are only 1 bp long, I need to merge adjacent windows together
def clipper_to_rMAPS(clipper_in):
    '''This function will parse clipper output to rMAPS function output.
    Input = clipper output file'''
    import pandas as pd
    import pybedtools as pbt
    clipper_header = ['chr','start','stop','cluster_name','fdr','strand','apex_start','apex_stop']
    rMAPS_header = ['chr','start','stop','cluster_name','read_count','strand','fdr']
    clipper_df = pd.read_csv(clipper_in,sep='\t',header=None,names=clipper_header)
    clipper_df['read_count'] = clipper_df.cluster_name.apply(lambda x: x.split('_')[-1])
    clipper_df = clipper_df[rMAPS_header]
    clipper_df['cluster_name'] = clipper_df.cluster_name.apply(lambda x: x.replace('.','_'))
    clipper_df.to_csv(clipper_in+'_rMAPS',sep='\t',index=False,header=False)
    clipper_bt = pbt.BedTool.from_dataframe(clipper_df)
    clipper_bt_merge = clipper_bt.merge(c=[4,5,6,7],o=['distinct','sum','distinct','mean'])
    clipper_bt_merge.saveas(clipper_in+'_merge_rMAPS')
    return

In [110]:
#now use this function to do the other clippers with normal clipper out and pre_mRNA
clipper_to_rMAPS('/media/sam/Data1/hnRNPM_iCLIP/iCLIP_analysis/getting_clipper_to_work/clipper_runs/\
hnM_DMSO_clipper/dmso_clipper_out_premrna')
clipper_to_rMAPS('/media/sam/Data1/hnRNPM_iCLIP/iCLIP_analysis/getting_clipper_to_work/clipper_runs/\
hnM_TAM_clipper/tam_clipper_out_premrna')

In [118]:
#now deal with the union.bg
#Remember for this one we want files with differential binding between the two.
union_df = pd.read_csv('/media/sam/Data1/hnRNPM_iCLIP/iCLIP_analysis/normalization_and_correlation_between_RT_stops/hnM_DMSO_vs_TAM_ryan/hnM_DMSO_TAM_binding_and_gene_expression_dataframe',sep='\t')
union_df.head()

,chrom,start,end,hnM_DMSO,hnM_TAM,hnM_DMSO_1,hnM_TAM_1,log2ratio_TAM_DMSO,ens_gene,gene,gene_coords,twist_d0,twist_d14,twist_d0_1,twist_d14_1,log2ratio_d14_d0
0,chr1,251639,251647,14.219565,0.000000,15.219565,1.000000,-3.927855,ENSG00000228463,AP006222.2,chr1:227614-267253,0.040763,0.117389,1.040763,1.117389,0.102490
1,chr1,566242,566254,112.662709,135.544767,113.662709,136.544767,0.264615,ENSG00000230021,RP5-857K21.4,chr1:536815-755214,0.000000,0.000000,1.000000,1.000000,0.000000
2,chr1,566269,566270,8.750502,0.000000,9.750502,1.000000,-3.285476,ENSG00000230021,RP5-857K21.4,chr1:536815-755214,0.000000,0.000000,1.000000,1.000000,0.000000
3,chr1,566278,566279,0.000000,7.041287,1.000000,8.041287,3.007426,ENSG00000230021,RP5-857K21.4,chr1:536815-755214,0.000000,0.000000,1.000000,1.000000,0.000000
4,chr1,722062,722063,0.000000,7.041287,1.000000,8.041287,3.007426,ENSG00000237491,RP11-206L10.9,chr1:536815-755214,0.166419,0.137282,1.166419,1.137282,-0.036496


all_possible_hnRNPM_pentamers
CLIPMap.test.peaks.txt
clipper_to_rMAPS.py
clipper_to_rMAPS.py~
dmso_clipper_out_premrna_rMAPS
hnM_DMSO_clipper_out_rMAPS
hnM_TAM_clipper_out_rMAPS
parse_clipper_to_rMAPS.ipynb
rMATS.3.0.9_0.0001_cutoff_CC1_CC2_motif_analysis/
rMATS.3.0.9_0.05_cutoff_CC1_CC2_motif_analysis/
rMATS.3.2.2_CC1_CC2_motif_analysis/
rMATS.3.2.2_CC1_CC2_TAM_clip_analysis /
rMATS.3.2.2_CC3_CC4_motif_analysis/
rMATS.3.2.2_CC3_CC4_TAM_clip_analysis/
rMATS.3.2.2_HH5_HH6_DMSO_clip_analysis/
rMATS.3.2.2_HH5_HH6_motif_analysis/
rMATS.3.2.2_SEH-1_SEH-2_DMSO_clip_analysis/
rMATS_3.2.2_SEH-1_SEH-2_TAM_clip_analysis/
tam_clipper_out_premrna_rMAPS


chr1	251624	251660	ENSG00000228463_4_0_13	13	-	5.29130327463e-12
 chr1	564580	564587	ENSG00000225972_1_0_7	7	+	0.000262761079095
 chr1	564587	564607	ENSG00000225972_1_1_7	7	+	0.00510839678809
 chr1	565338	565345	ENSG00000225630_1_0_7	7	+	0.00023438364786
 chr1	565345	565367	ENSG00000225630_1_1_7	7	+	0.00598069914663
 chr1	565665	565672	ENSG00000230021_2_0_5	5	-	0.000608837058219
 chr1	565672	565696	ENSG00000230021_2_1_5	5	-	0.000640902134525
 chr1	565717	565724	ENSG00000225630_1_2_8	8	+	4.0205219743e-05
 chr1	565724	565747	ENSG00000225630_1_3_8	8	+	0.00202998044172
 chr1	565819	565874	ENSG00000225630_1_4_23	23	+	8.81711563404e-10
 

chr1	251624	251660	ENSG00000228463_4_0_13	13	-	5.291303275e-12
 chr1	564580	564607	ENSG00000225972_1_0_7,ENSG00000225972_1_1_7	14	+	0.002685578934
 chr1	565338	565367	ENSG00000225630_1_0_7,ENSG00000225630_1_1_7	14	+	0.003107541397
 chr1	565665	565696	ENSG00000230021_2_0_5,ENSG00000230021_2_1_5	10	-	0.0006248695964
 chr1	565717	565747	ENSG00000225630_1_2_8,ENSG00000225630_1_3_8	16	+	0.001035092831
 chr1	565819	565914	ENSG00000225630_1_4_23,ENSG00000225630_1_5_38	61	+	4.408557817e-10
 chr1	565977	566008	ENSG00000225630_1_6_9,ENSG00000225630_1_7_9	18	+	0.0003296087092
 chr1	566227	566285	ENSG00000230021_2_2_103,ENSG00000230021_2_3_103,ENSG00000230021_2_4_111,ENSG00000230021_2_5_8,ENSG00000230021_2_6_8	333	-	4.800777729e-07
 chr1	566466	566525	ENSG00000237973_1_0_25	25	+	7.117587226e-07
 chr1	566546	566578	ENSG00000237973_1_1_14	14	+	0.000236221425
 